# CMAPSS using Deep Convolutional Neural Networks
This notebook contains a minimal example of DCNN for the CMAPSS datasets, including
1. Updated and refactored data preprocessing scripts
2. CNN architecture and training steps

```
Author: CSN
Last modified: 20221102
```

In [ ]:
# Load the "autoreload" extension so that code can change
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Import all relevant libraries
import pandas as pd
import matplotlib.pyplot as plt

from utils.utils import (clean_train_dataf, clean_test_dataf,
                         scale_train_dataf, scale_test_dataf,
                         lag_dataframe, shape_dataframe_to_sequence)

import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

In [ ]:
filename = './data/train_FD001.tar.gz'

# Set the column names for the raw CMAPSS data
columns = ['machine_number', 'uptime', 'setting_1', 'setting_2', 'setting_3']
sensor_measurements = [f'sensor_{i:02d}' for i in range(1, 25)]
columns += sensor_measurements

# Read plain text file
df = pd.read_csv(filename, sep=" ", names=columns, index_col=False)

# Drop dummy columns
df.drop(columns=['sensor_22', 'sensor_23', 'sensor_24'], inplace=True)

In [ ]:
# Supplementary cell to write data for PCS-CMAPSS
def add_train_RUL(grp):
    # Apply clipping to RUL based on uptime
    grp['RUL'] = (grp['uptime'].max() - grp['uptime'] + 1)
    # Apply clipping to RUL
    grp['RUL'].clip(upper=125, inplace=True)

    return grp

# Verify the number of columns in the training df
(df
  .sort_values(by=['machine_number', 'uptime'], axis=0)
  .groupby('machine_number', group_keys=True)
  .apply(add_train_RUL)
).to_csv('./data/train_FD001.csv', index=False)
pd.read_csv('./data/train_FD001.csv').shape

In [ ]:
# Use the profiler to remove constant values
from pandas_profiling import ProfileReport
profile = ProfileReport(df)
# profile.to_file("FD001.html")

Next we load or recomputed the rejected features. This list of rejected features is quite well known throughout literature. The main reasons why they are selected is because they contain constants, whereas other features contain some form of trends.

In [ ]:
rejected_feature_fname = './data/rejected_features.txt'
if os.path.exists(rejected_feature_fname):
    # Check if has feature selection has been done and load the file
    with open(rejected_feature_fname, 'r') as f:
        rejected_features = f.read().splitlines()
else:
    # Get list of rejected variables due to constant values
    rejected_features = list(profile.get_rejected_variables())
    # Remove sensor_06 which is also constant. Comes from viewing the data.
    rejected_features += ['setting_1', 'setting_2', 'sensor_06']
    # Write to file
    with open(rejected_feature_fname, 'w') as f:
        for feature in rejected_features:
            f.write(f"{feature}\n")

Now we load the test datasets for the final scoring.

In [ ]:
# Now load the test set and apply the same preprocessing
df_test = pd.read_csv('./data/test_FD001.tar.gz', sep=" ", names=columns, index_col=False)

# Drop dummy columns
df_test.drop(columns=['sensor_22', 'sensor_23', 'sensor_24'], inplace=True)

# Load the y_test
y_test = pd.read_csv('./data/RUL_FD001.tar.gz', names=['RUL'])
y_test['machine_number'] = y_test.index + 1

# Join the test X and label dataframes
df_test = df_test.join(y_test.set_index('machine_number'), on='machine_number')

In [ ]:
# Supplementary cell to write data for PCS-CMAPSS demo
def add_test_RUL(grp):
    grp['RUL'] += grp['uptime'].max() - grp['uptime']
    # Apply clipping to RUL
    grp['RUL'].clip(upper=125, inplace=True)
    return grp

# Verify the number of columsn in the test set
(df_test
  .sort_values(['machine_number', 'uptime'], axis=0)
  .groupby('machine_number', group_keys=True)
  .apply(add_test_RUL)
).to_csv('./data/test_FD001.csv', index=False)
pd.read_csv('./data/test_FD001.csv').shape

In the next cell we preprocess all the data using the `pandas` pipeline functionality.

In [ ]:
num_lags = 30

# Run pre-processing on train set
clean_df = (df
            .pipe(clean_train_dataf, rejected_features=rejected_features)
            .drop(columns=['uptime'])
            )
# Split this pipe step to store scaler for later reuse
clean_df, scaler = clean_df.pipe(scale_train_dataf)
# clean_df = clean_df.pipe(lag_dataframe, num_lags=num_lags)

# Run pre-processing on test set
clean_df_test = (df_test
                 .pipe(clean_test_dataf, rejected_features=rejected_features)
                 .drop(columns=['uptime'])
                )
clean_df_test = clean_df_test.pipe(scale_test_dataf, scaler=scaler)
# clean_df_test = clean_df_test.pipe(lag_dataframe, num_lags=num_lags)

Now we construct our train, test, and validation sets.

In [ ]:
x_train, y_train = shape_dataframe_to_sequence(clean_df, num_lags)
x_test, y_test = shape_dataframe_to_sequence(clean_df_test, num_lags)

# Make sure input array has shape (num_lags, num_sensors, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32 )
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float32 )
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)

# Training and evaluation
This section contains a basic MLP model and training procedure

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    # Define an rms loss function
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def scheduler(epoch, lr):
    # Define a custom scheduler to vary learning rate based on epoch
    # Based on: https://doi.org/10.1016/j.ress.2017.11.021, page 5
    if epoch < 100:
      return lr
    else:
      lr = 1e-4
      return lr


In [ ]:
# Create the model using the functional API approach
filters, kernel_size = 10, 10  # num of convnet filters, size of filter kernel
input_shape = np.shape(x_train)[1:]
inputs = keras.Input(shape=input_shape)
x = layers.Conv1D(filters, kernel_size, padding="same", activation="tanh")(inputs)
x = layers.Conv1D(filters, kernel_size, padding="same", activation="tanh")(x)
x = layers.Conv1D(filters, kernel_size, padding="same", activation="tanh")(x)
x = layers.Conv1D(filters, kernel_size, padding="same", activation="tanh")(x)
x = layers.Conv1D(1, 3, padding="same", activation="tanh")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="regress_model")

model.compile(
    loss=root_mean_squared_error,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mse', 'mae', 'mape']
)

model.summary()

In [ ]:
# Create callback for scheduling learning rate
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(
    x=x_train,
    y=y_train,
    epochs=10,
    # Suppress logging.
    # verbose=0,
    validation_data=(x_val, y_val),
    callbacks=[callback],)

In [ ]:
def plot_loss(history):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].plot(history.history['loss'], label='train_loss')
    axs[0].plot(history.history['val_loss'], label='val_loss')
    axs[1].plot(history.history['mse'], label='train_mse')
    axs[1].plot(history.history['val_mse'], label='val_mse')
    for ax in axs:
        ax.set_xlabel('Epoch')
        ax.legend()
        ax.grid(True)
    axs[0].set_ylabel(' RUL')
    axs[1].set_ylabel('MSE RUL')


In [ ]:
plot_loss(history)
plt.show()

# Visualise prediction on the test datasets

Here, we run a prediction on the test set, which the model has not seen before. We also visualise a subset of the predictions against the ground truth as a sanity check.

In [ ]:
ind_ = 3500
plt.figure(figsize=(14, 3))
plt.plot(model.predict(x_test[:ind_, :]))
plt.plot(y_test[:ind_], '--')
plt.show()

# Evaluate prediction on the test datasets

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)
print("test loss, test mse, test mae, test mape:", results)
print(f"test rmse {np.sqrt(results[1]):.4f}")


In [ ]:
import netron
model.save('./data/convnet_li_tanh.h5')
netron.start('./data/convnet_li_tanh.h5')

## Note
It is a good practice to re-train the model multiple times and evaluate the mean and standard deviation of the rmse. This is due to the randomness of the initialisations as well as the dropout rate.